In [1]:
from music21 import *
import os
from fractions import *
import numpy as np
from midi_note import MIDINote
import json

In [2]:
#Global Variables

PATH = "POP909/"
LIMIT = 2
MIDI_NOTE_DIRECTORY = "midi_notes_list.json"


In [3]:
def parse_raw_notes(file):
    print("Loading Music File:", file)
    raw_notes = []

    midi_data = converter.parse(file)

    for part in midi_data.parts:
        if part.partName == 'MELODY':
            midi_elements = part.recurse()
            for element in midi_elements:
                if isinstance(element, note.Note) or isinstance(element, note.Rest):
                    note_duration = duration.Duration()
                    note_duration.quarterLength = element.quarterLength

                    if isinstance(element, note.Rest):
                        raw_note = str(element.name)
                    else:
                        raw_note = str(element.pitch)

                    raw_notes.append(MIDINote(raw_note, str(note_duration.type), str(element.quarterLength)))

    return np.array(raw_notes)

In [4]:
def get_midi_file_paths():
    path = PATH
    limit = LIMIT
    midi_files_path = []
    for index, i in enumerate(os.listdir(path)):
        if index == limit:
            break
        for j in os.listdir(path + i):
            midi_path = (i + '/' + j)
            if midi_path.endswith(".mid"):
                midi_files_path.append(midi_path)
    return np.array(midi_files_path)

In [5]:
def load_data():
    midi_files_path = get_midi_file_paths()

    raw_notes_list = []

    for midi_path in midi_files_path:
        parsed_midi_notes = parse_raw_notes(PATH + midi_path)
        print(parsed_midi_notes.shape)
        raw_notes_list.append(parsed_midi_notes)

    return np.array(raw_notes_list).reshape((LIMIT, 1))

In [8]:
def to_json(midi_notes_list):
    json_file = {}
    for index, midi_notes in enumerate(midi_notes_list):
        notes = midi_notes[0]

        json_file[index] = [_note.as_map for _note in notes]

    with open(MIDI_NOTE_DIRECTORY) as f:
        data = json.load(f)
        data.update(json_file)
    with open(MIDI_NOTE_DIRECTORY, 'w') as f:
        json.dump(data, f)

In [ ]:
# to work tomorrow
def to_midi_note():
    pass


In [10]:
all_notes = load_data()

to_json(all_notes)
all_notes.shape

Loading Music File: POP909/001/001.mid
(549,)
Loading Music File: POP909/002/002.mid
(552,)


C:\Users\sheesh\AppData\Local\Temp\ipykernel_13184\3037971445.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(raw_notes_list).reshape((LIMIT, 1))


(2, 1)